<left>FINM 33150 - Quantitative Trading Strategies</left>
<left>Winter 2023</left>
<br>
<h1><center> Homework 4: Accumulation Opportunity </center></h1>
<center>Due - 23:00 [CST] February 2nd, 2023</center>
<br>
<h3>Ki Hyun</h3>
<h3>Student ID: 12125881</h3>

<h5> Imports </h5>

In [1]:
%matplotlib inline

In [2]:
import os
import datetime
import pickle
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import numpy as np
import scipy as sp
import seaborn as sns
from enum import Enum

<h5> Constants </h5>

In [3]:
# constants needed for grabbing and cleaning data
data_home_dir = "C:\\Users\\kwhyu\\OneDrive - The University of Chicago\\2023-1 Winter\\FINM " \
                "33150\\FINM-33150-W23\\Data\\Crypto\\"
pairs = ['BTC-USD', 'ETH-USD', 'ETH-BTC']
years = ['2021', '2022', '2023']
# constants needed for trading simulation
reaction_time = int(0.1 * (10 ** 9)) # in nano seconds
currency_crypto_cost = 50 * (10 ** (-4))
crypto_crypto_cost = 10 * (10 ** (-4))
p = 0.05 # 5% participation rate
one_bitcoin = 10**9

<h5> Helper Functions </h5>

In [4]:
class Order(Enum):
    Sell = -1
    Buy = 1

In [5]:
def read_data(pair, year):
    global data_home_dir
    # creating directory for the pair and year
    book_dir = f'book_narrow_{pair}_{year}.delim.gz'
    trades_dir = f'trades_narrow_{pair}_{year}.delim.gz'
    # retrieving book data:
    print(">>> Retrieving Book data for ", pair, " for the year ", year, "...")
    try:
        book = pd.read_table(os.path.join(data_home_dir, book_dir))
        print(">>> Success!")
    except:
        print(">>> Book data for ", pair, " for the year ", year, " does not exist")
        book = None
    # retrieving trades data:
    print(">>> Retrieving Trades data for ", pair, " for the year ", year, "...")
    try:
        trades = pd.read_table(os.path.join(data_home_dir, trades_dir))
        print(">>> Success!")
    except:
        print(">>> Trades data for ", pair, " for the year ", year, " does not exist")
        return book, None, pair, year, None
    # indexing trades data by Time and Nano second
    trades['Time'] = (trades['timestamp_utc_nanoseconds'] / 10**9).apply(int)\
        .apply(lambda x: datetime.datetime.fromtimestamp(x))
    trades['Nano second'] = trades['timestamp_utc_nanoseconds'] % 10**9
    trades = trades.set_index(['Time', 'Nano second']).sort_index().drop(columns = {'received_utc_nanoseconds'})
    # sorting into buy and sell data
    sell = trades[trades['Side'] < 0]
    buy = trades[trades['Side'] > 0]

    return book, buy, pair, year, sell

In [6]:
def clean_trading_data(raw_df, side, reaction_time):
    df = raw_df[side.value].copy()
    pair = raw_df[2]
    year = raw_df[3]
    if side == Order.Buy:
        detail = "Buy"
    else:
        detail = "Sell"
    print(">>> Cleaning", pair, "for the year", year, detail, "Trades data for VWAP Algorithm based on reaction time",
          reaction_time/(10**9), " seconds...")
    df['hold_time_nanoseconds'] = (df['timestamp_utc_nanoseconds'].shift(-1).fillna(method = 'ffill') -
                                   df['timestamp_utc_nanoseconds']).apply(int)
    df = df.drop(columns = {'timestamp_utc_nanoseconds', 'Side'})
    df = df[df['hold_time_nanoseconds'] > 0]

    temp_l = []
    interval = int(0)
    for trade in df.index:
        interval += df.loc[trade, 'hold_time_nanoseconds']
        if interval >= reaction_time:
            temp_l.append(True)
            interval = int(0)
        else:
            temp_l.append(False)
    df['qualify'] = temp_l
    print(">>> Success!")
    return df

In [7]:
def data_factory(pairs, years, avoid_file_load = False):
    if not avoid_file_load:
        dir = r'./data/full_data.pkl'
        with open(dir, 'rb') as f:
            try:
                ret_dict = pickle.load(f)
                print(">>> Found data...")
                return ret_dict
            except:
                print(">>> No data found")
                print(">>> Proceeding to compute the data...")

    global reaction_time
    ret_dict = {}
    for pair in pairs:
        for year in years:
            raw_df = read_data(pair, year)
            if raw_df[1] is None:
                buy = None
                sell = None
            else:
                buy = clean_trading_data(raw_df, Order.Buy, reaction_time)
                sell = clean_trading_data(raw_df, Order.Sell, reaction_time)

            ret_dict[(pair, year)] = [raw_df[0], buy, sell]

    if not avoid_file_load:
        dir = r'./data/full_data.pkl'
        with open(dir, 'wb') as f:
            pickle.dump(ret_dict, f)

    return ret_dict

In [8]:
def VWAP_participation(df_cleaned, Q, p, start_time,
                       side, transaction_cost):
    df = df_cleaned.loc[start_time:].copy()

    if side == Order.Buy:
        f = False
    else:
        f = True

    accumulated = 0
    indexes = df[df['qualify']].index
    i = 0
    ret_df = {'datetime': [],
              'Nano seconds': [],
              'PriceMillionths': [],
              'SizeBillionths': []}

    while(accumulated < Q and i < len(indexes)):
        if i == 0:
            temp = df.loc[:indexes[i]].sort_values(by = ['PriceMillionths'], ascending = f).iloc[0]
        else:
            temp = df.loc[indexes[i-1]:indexes[i]].iloc[1:].sort_values(by = ['PriceMillionths'], ascending = f).iloc[0]

        ret_df['datetime'].append(temp.name[0])
        ret_df['Nano seconds'].append(temp.name[1])
        ret_df['PriceMillionths'].append(temp['PriceMillionths'])
        ret_df['SizeBillionths'].append(temp['SizeBillionths'] * p)

        accumulated += temp['SizeBillionths'] * p
        i += 1

    ret_df = pd.DataFrame(ret_df).set_index(['datetime', 'Nano seconds'])
    ret_df['NotionalMillionths'] = ret_df['PriceMillionths'] * ret_df['SizeBillionths'] / (10**9)
    ret_df['TradingCostsMillionths'] = ret_df['NotionalMillionths'] * transaction_cost
    ret_df['VWAPMillionths'] = (ret_df['NotionalMillionths'].cumsum() / ret_df['SizeBillionths'].cumsum() * (10**9))\
        .apply(int)

    return ret_df

In [9]:
def total_participation_opportunities(full_dict, p, currency_crypto_cost, crypto_crypto_cost):

    for pair, year in full_dict.keys():
        if pair == 'ETH-BTC':
            transaction_cost = crypto_crypto_cost
        else:
            transaction_cost = currency_crypto_cost
        df_Buy = full_dict[(pair, year)][Order.Buy.value]
        dir = r'./data/' + pair + '_' + year + '_total_buy_opportunities.csv'
        if os.path.exists(dir):
            print('>>>', dir, 'in data file')
        else:
            print('>>>', dir, 'not in data file')
            print('>>> Computing Data...')
            VWAP_participation(df_Buy, np.infty, p, df_Buy.index[0][0], Order.Buy, transaction_cost).to_csv(dir)
            print('>>> Success!')
        df_Sell = full_dict[(pair, year)][Order.Sell.value]
        dir = r'./data/' + pair + '_' + year + '_total_sell_opportunities.csv'
        if os.path.exists(dir):
            print(dir, 'in data file')
        else:
            print('>>>', dir, 'not in data file')
            print('>>> Computing Data...')
            VWAP_participation(df_Sell, np.infty, p, df_Sell.index[0][0], Order.Sell, transaction_cost).to_csv(dir)
            print('>>> Success!')

In [10]:
def read_pair_data(pair, year):
    # buy data
    dir_buy = r'./data/' + pair + '_' + year + '_total_buy_opportunities.csv'
    buy_df = pd.read_csv(dir_buy)
    buy_df.datetime = pd.to_datetime(buy_df.datetime)
    buy_df = buy_df.set_index(['datetime', 'Nano seconds'])
    # sell data
    dir_sell = r'./data/' + pair + '_' + year + '_total_sell_opportunities.csv'
    sell_df = pd.read_csv(dir_sell)
    sell_df.datetime = pd.to_datetime(sell_df.datetime)
    sell_df = sell_df.set_index(['datetime', 'Nano seconds'])
    return buy_df, sell_df

In [11]:
def accumulation_analytics(dfs, Q):
    # copying original data
    df = dfs.copy()
    # creating cumulative sum data
    df_cumulative = df.copy()
    df_cumulative['SizeBillionths'] = df_cumulative['SizeBillionths'].cumsum().shift().fillna(0.0)
    df_cumulative['NotionalMillionths'] = df_cumulative['NotionalMillionths'].cumsum().shift().fillna(0.0)
    df_cumulative['TradingCostsMillionths'] = df_cumulative['TradingCostsMillionths'].cumsum().shift().fillna(0.0)
    # Q based cutoffs
    df_cumulative['Q_count'] = (df_cumulative['SizeBillionths'] / Q).apply(int) + 1
    df_cumulative['Starters'] = df_cumulative['Q_count'] - df_cumulative['Q_count'].shift(2).fillna(0)
    df_cumulative['Q_wise'] = df_cumulative['SizeBillionths'] - df_cumulative['Q_count'].shift().fillna(1) * Q
    # arrival times data
    df_arrivals = df_cumulative[(df_cumulative['Starters'] > 0) & (df_cumulative['Q_wise'] < 0)]
    df_arrivals = df_arrivals.drop(df_arrivals.index[1], axis = 0).reset_index()
    df_arrivals.rename(columns = {'datetime': 'Start Time',
                                  'Nano seconds': 'Start Nano',
                                  'PriceMillionths': 'Arrival Price Millionths',
                                  'SizeBillionths': 'bought size',
                                  'NotionalMillionths': 'bought notional',
                                  'TradingCostsMillionths': 'bought costs',
                                  'VWAPMillionths': 'Arrival VWAP'},
                       inplace = True)
    df_arrivals = df_arrivals.drop(columns = ['Q_count', 'Starters', 'Q_wise'])
    # end points data
    df_endpoints = df_cumulative[(df_cumulative['Starters'] > 0) & (df_cumulative['Q_wise'] > 0)].reset_index()
    df_endpoints.rename(columns = {'datetime': 'End Time',
                                  'Nano seconds': 'End Nano',
                                  'SizeBillionths': 'Accumulated Billionth',
                                  'NotionalMillionths': 'Notional Millionth',
                                  'TradingCostsMillionths': 'Transaction Cost Millionth',
                                  'VWAPMillionths': 'VWAP Millionths'},
                       inplace = True)
    df_endpoints = df_endpoints.drop(columns = ['Q_count', 'Starters', 'Q_wise'])
    # merging for singular transaction
    ret_df = df_arrivals.merge(df_endpoints, how = 'inner', left_index = True, right_index = True)
    # calculating analytic statistics
    ## Accumulated
    ret_df['Accumulated Billionth'] = ret_df['Accumulated Billionth'] - ret_df['bought size']
    ## Notional
    ret_df['Notional Millionth'] = ret_df['Notional Millionth'] - ret_df['bought notional']
    ## VWAP
    ret_df['VWAP Millionths'] = ret_df['Notional Millionth'] / ret_df['Accumulated Billionth'] * (10**9)
    ret_df['VWAP Millionths'] = ret_df['VWAP Millionths'].apply(int)
    ## Transaction cost
    ret_df['Transaction Cost Millionth'] = ret_df['Transaction Cost Millionth'] - ret_df['bought costs']
    ## Duration
    ret_df['Approximate Duration Time'] = ret_df['End Time'] - ret_df['Start Time']
    ret_df['Duration Nano'] = ret_df['End Nano'] - ret_df['Start Nano']
    ret_df['Duration Nano'] = (ret_df['Approximate Duration Time'].apply(datetime.timedelta.total_seconds)* 10**9 +
                               ret_df['Duration Nano']).apply(int)
    ## Ratios
    ret_df[r'$\frac{Notional}{Time}$'] = (ret_df['Notional Millionth']/ ret_df['Duration Nano']) * 10**3
    ret_df[r'$\frac{Accumulation}{Time}$'] = ret_df['Accumulated Billionth'] / ret_df['Duration Nano']
    # dropping unnecessary columns
    ret_df = ret_df.drop(columns = ['bought size', 'bought notional', 'bought costs', 'Arrival VWAP',
                                    'PriceMillionths'])

    return ret_df

In [12]:
def report_summary(df, side, year):
    if side == Order.Buy:
        phrase = ["accumulated", "depletion", "accumulation", "accumulating"]
    else:
        phrase = ["liquidated", "accumulation", "liquidation", "liquidating"]

    mu = df[r'$\frac{Accumulation}{Time}$'].mean()
    sigma = df[r'$\frac{Accumulation}{Time}$'].std()
    stat = (1/(15*60) - mu)/sigma

    print("There are total of", len(df), "transactions in the year", year, "data where 1 Bitcoin was", phrase[0])
    print("The average duration time was", df['Duration Nano'].mean()/(60*10**9), "minutes")
    print("The average VWAP for the transaction was", df['VWAP Millionths'].mean()/(10**6), "US Dollars")
    print("The average transaction cost was", df['Transaction Cost Millionth'].mean()/(10**6), "US Dollars")
    print("The average", phrase[1], "of capital for the transaction was",
          df[r'$\frac{Notional}{Time}$'].mean(), "US Dollars per second")
    print("The average", phrase[2], "rate was", mu, "Bitcoins per second")
    print("The standard deviation of", phrase[2], "rate was", sigma, "Bitcoins per second")
    print("Using Normal Approximation, the likelihood of", phrase[3], "1 Bitcoin under 15 minutes is approximately",
          ((1 - sp.stats.norm.cdf(stat)) * 100).round(4), "%")

    return None

<h2> 2. Data </h2>

<h3> 2-1. Raw Data </h3>

The raw data for this analysis was given by Dr. Boonstra

<h3> 2-2. Data Processing </h3>

In [13]:
# reading or computing qualifying trades from the raw data
data_dict = data_factory(pairs, years)

>>> Found data...


In [14]:
# reading or computing and saving total accumulation opportunities data
total_participation_opportunities(data_dict, p, currency_crypto_cost, crypto_crypto_cost)

>>> ./data/BTC-USD_2021_total_buy_opportunities.csv in data file
./data/BTC-USD_2021_total_sell_opportunities.csv in data file
>>> ./data/BTC-USD_2022_total_buy_opportunities.csv in data file
./data/BTC-USD_2022_total_sell_opportunities.csv in data file
>>> ./data/BTC-USD_2023_total_buy_opportunities.csv in data file
./data/BTC-USD_2023_total_sell_opportunities.csv in data file
>>> ./data/ETH-USD_2021_total_buy_opportunities.csv in data file
./data/ETH-USD_2021_total_sell_opportunities.csv in data file
>>> ./data/ETH-USD_2022_total_buy_opportunities.csv in data file
./data/ETH-USD_2022_total_sell_opportunities.csv in data file
>>> ./data/ETH-USD_2023_total_buy_opportunities.csv in data file
./data/ETH-USD_2023_total_sell_opportunities.csv in data file
>>> ./data/ETH-BTC_2021_total_buy_opportunities.csv in data file
./data/ETH-BTC_2021_total_sell_opportunities.csv in data file
>>> ./data/ETH-BTC_2022_total_buy_opportunities.csv in data file
./data/ETH-BTC_2022_total_sell_opportunities.c

In [15]:
BTC_USD_2021 = read_pair_data('BTC-USD', '2021')
BTC_USD_2022 = read_pair_data('BTC-USD', '2022')
BTC_USD_2023 = read_pair_data('BTC-USD', '2023')

The first step of data processing was to clean the raw data to identify qualifying orders.
By qualifying orders, it refers to buy or sell orders that are not too big and orders that do not happen too fast.

Orders that are too big would clear multiple levels on the other side (bid book for buy orders and ask book for sell
orders). This could be identified in the raw data through transactions that are precisely 0 nano-seconds apart. The
last orders of the series of orders that were 0 nano-seconds apart were identified as qualifying orders.

Orders that are too fast would happen within the reaction time. If there are multiple orders happening within the
reaction time, the conservative assumption was made such that one would have only been able to participate in the worst
of those orders (i.e., highest price for buy and lowest price for sell). The worst orders in each series of orders that
happened within the reaction time were identified as qualifying orders.

The reaction time was set to 0.1 seconds and the raw data was initially cleaned for these qualifying orders.
Qualifying orders were filtered for each of the buy and sell side of the 3 pairs across 3 years (2021 - 2023)
Compiling the data took approximately 20 minutes.

The second step of the data processing was to simulate the VWAP participation.
For each qualifying orders, one was assumed to have participated with the participation rate ($p$) of the order's size.
For the VWAP participation algorithm, the target quantity ($Q$), start time ($\tau_s$), and participation rate ($p$)
were set as parameters. The algorithm will return a dataframe of participated trades either until the target quantity
($Q$) as been reached, or until the algorithm runs through all the qualifying orders.

The last step of the data processing was to identify all the possible participation trades for a given year for both
buy and sell sides across three different pairs. For each pair and year, the cleaned data (only including qualifying
orders) was passed on to the VWAP participation algorithm with the target quantity set as infinity ($Q = \infty$) so
that the algorithm runs through all the qualifying orders.
Compiling the data took approximately 4 hours.

The transaction fees for each participation, in terms of the notional trade amount, were 50 basis points (0.5%) for
transactions between crypto-tokens and traditional currencies; 10 basis points (0.1%) for transactions between
crypto-tokens.

<h3> 2-3. Data Description </h3>

In [16]:
BTC_USD_2021[Order.Buy.value - 1].head(10)

PriceMillionths  SizeBillionths  \
datetime            Nano seconds                                    
2021-04-10 16:28:53 470961000         59122080000        189904.5   
                    596148000         59119960000        126228.5   
2021-04-10 16:28:54 64537000          59119960000         27776.5   
                    835128000         59123810000        984083.5   
2021-04-10 16:28:55 30223000          59123810000         11798.0   
                    577343000         59123810000       3785645.0   
2021-04-10 16:29:00 30069000          59123580000      16450622.0   
2021-04-10 16:29:02 519128000         59121540000         81632.0   
                    810133000         59117620000          5060.5   
2021-04-10 16:29:03 5146000           59117620000         81630.0   

                                  NotionalMillionths  TradingCostsMillionths  \
datetime            Nano seconds                                               
2021-04-10 16:28:53 470961000           1.122755e+07            5.613775e+04   
                    596148000           7.462624e+06            3.731312e+04   
2021-04-10 16:28:54 64537000            1.642146e+06            8.210728e+03   
                    835128000           5.818277e+07            2.909138e+05   
2021-04-10 16:28:55 30223000            6.975427e+05            3.487714e+03   
                    577343000           2.238218e+08            1.119109e+06   
2021-04-10 16:29:00 30069000            9.726197e+08            4.863098e+06   
2021-04-10 16:29:02 519128000           4.826210e+06            2.413105e+04   
                    810133000           2.991647e+05            1.495824e+03   
2021-04-10 16:29:03 5146000             4.825771e+06            2.412886e+04   

                                  VWAPMillionths  
datetime            Nano seconds                  
2021-04-10 16:28:53 470961000        59122080000  
                    596148000        59121233506  
2021-04-10 16:28:54 64537000         59121130649  
                    835128000        59123116130  
2021-04-10 16:28:55 30223000         59123122240  
                    577343000        59123630219  
2021-04-10 16:29:00 30069000         59123591929  
2021-04-10 16:29:02 519128000        59123584195  
                    810133000        59123582802  
2021-04-10 16:29:03 5146000          59123560417

In [17]:
BTC_USD_2021[Order.Sell.value].head(10)

PriceMillionths  SizeBillionths  \
datetime            Nano seconds                                    
2021-04-10 16:28:52 515484000         59125070000        406122.5   
2021-04-10 16:28:53 74765000          59119950000        849794.5   
                    613354000         59119950000        656146.5   
2021-04-10 16:28:54 195667000         59119950000        891207.0   
                    716996000         59119950000       1250000.0   
2021-04-10 16:28:55 71608000          59119950000        132392.0   
                    358280000         59119950000        891184.5   
                    987817000         59121860000        891028.5   
2021-04-10 16:28:56 830404000         59121860000        352749.0   
2021-04-10 16:28:57 426911000         59119960000          9283.0   

                                  NotionalMillionths  TradingCostsMillionths  \
datetime            Nano seconds                                               
2021-04-10 16:28:52 515484000           2.401202e+07           120060.106205   
2021-04-10 16:28:53 74765000            5.023981e+07           251199.041751   
                    613354000           3.879135e+07           193956.741363   
2021-04-10 16:28:54 195667000           5.268811e+07           263440.566398   
                    716996000           7.389994e+07           369499.687500   
2021-04-10 16:28:55 71608000            7.827008e+06            39135.042102   
                    358280000           5.268678e+07           263433.915404   
                    987817000           5.267926e+07           263396.311165   
2021-04-10 16:28:56 830404000           2.085518e+07           104275.884966   
2021-04-10 16:28:57 426911000           5.488106e+05             2744.052943   

                                  VWAPMillionths  
datetime            Nano seconds                  
2021-04-10 16:28:52 515484000        59125070000  
2021-04-10 16:28:53 74765000         59121605640  
                    613354000        59121037488  
2021-04-10 16:28:54 195667000        59120691757  
                    716996000        59120463004  
2021-04-10 16:28:55 71608000         59120446778  
                    358280000        59120359574  
                    987817000        59120583594  
2021-04-10 16:28:56 830404000        59120654829  
2021-04-10 16:28:57 426911000        59120653810

The data above shows the first 10 rows of simulated Bitcoin and USD VWAP participation trade for the year 2021.
Each dataset has 5 columns: price in millionths, size in billionths, notional trade amount in millionths, trading
cost/transaction fee in millionths, and VWMAP in millionths.
The time up to seconds and nano-seconds were separated but were both set as indexes

The summary of Bitcoin-USD trade from the data is shown below.

In [18]:
trades = BTC_USD_2021[Order.Buy.value - 1].shape[0]
amount = BTC_USD_2021[Order.Buy.value - 1]['SizeBillionths'].sum() / 10**9
print("There were a total of", trades, "buys simulated where", amount, "Bitcoins were accumulated")

There were a total of 2528704 buys simulated where 3238.9367822165 Bitcoins were accumulated


In [19]:
trades = BTC_USD_2021[Order.Sell.value].shape[0]
amount = BTC_USD_2021[Order.Sell.value]['SizeBillionths'].sum() / 10**9
print("There were a total of", trades, "sells simulated where", amount, "Bitcoins were liquidated")

There were a total of 1013182 sells simulated where 2302.6818329885 Bitcoins were liquidated


<h2> 3. Analysis </h2>

This analysis will focus on the trades between Bitcoin and USD.

The standard for accumulation and liquidation would be 1 whole Bitcoin.

The analysis will be comparing the price, cost, and rate of accumulation/liquidation of 1 Bitcoin across the years
2021, 2022, and 2023.

Lastly, with the standard time-limit set as 15 minutes, and using normal approximation, this analysis will try and
quantify the likelihood of accumulating/liquidating 1 Bitcoin under the time-limit.

<h3> 3-1. Year 2021 </h3>

In [20]:
# year 2021
BTC_USD_2021_buys = accumulation_analytics(BTC_USD_2021[Order.Buy.value - 1], one_bitcoin)
BTC_USD_2021_sells = accumulation_analytics(BTC_USD_2021[Order.Sell.value], one_bitcoin)

In [21]:
BTC_USD_2021_buys

,Start Time,Start Nano,Arrival Price Millionths,End Time,End Nano,Accumulated Billionth,Notional Millionth,Transaction Cost Millionth,VWAP Millionths,Approximate Duration Time,Duration Nano,$\frac{Notional}{Time}$,$\frac{Accumulation}{Time}$
0,2021-04-10 16:28:53,470961000,59122080000,2021-04-10 16:44:10,898341000,1.004903e+09,5.940827e+10,2.970414e+08,59118394112,0 days 00:15:17,917427380000,64.755284,0.001095
1,2021-04-10 16:44:12,405529000,59118860000,2021-04-10 17:00:19,446786000,9.957953e+08,5.879374e+10,2.939687e+08,59041992013,0 days 00:16:07,967041257000,60.797551,0.001030
2,2021-04-10 17:00:19,655691000,59150000000,2021-04-10 17:11:24,799235000,9.992604e+08,5.920219e+10,2.960109e+08,59246006714,0 days 00:11:05,665143544000,89.006634,0.001502
3,2021-04-10 17:11:25,137427000,59300740000,2021-04-10 17:26:14,852082000,9.929717e+08,5.885975e+10,2.942987e+08,59276356748,0 days 00:14:49,889714655000,66.155757,0.001116
4,2021-04-10 17:26:15,867456000,59280010000,2021-04-10 17:47:29,282792000,1.000570e+09,5.932498e+10,2.966249e+08,59291210490,0 days 00:21:14,1273415336000,46.587297,0.000786
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3233,2021-04-26 18:27:59,244732000,53870840000,2021-04-26 18:39:20,494912000,1.025392e+09,5.525500e+10,2.762750e+08,53886707803,0 days 00:11:21,681250180000,81.108232,0.001505
3234,2021-04-26 18:39:21,843146000,53949080000,2021-04-26 18:49:25,658353000,9.836718e+08,5.307102e+10,2.653551e+08,53951958118,0 days 00:10:04,603815207000,87.892820,0.001629
3235,2021-04-26 18:49:26,254940000,54009720000,2021-04-26 18:59:54,249428000,9.966351e+08,5.383769e+10,2.691884e+08,54019455294,0 days 00:10:28,627994488000,85.729554,0.001587
3236,2021-04-26 18:59:54,830828000,54052500000,2021-04-26 19:06:23,472186000,9.968092e+08,5.382492e+10,2.691246e+08,53997217869,0 days 00:06:29,388641358000,138.495097,0.002565


In [22]:
report_summary(BTC_USD_2021_buys, Order.Buy, 2021)

There are total of 3238 transactions in the year 2021 data where 1 Bitcoin was accumulated
The average duration time was 7.157292011730492 minutes
The average VWAP for the transaction was 56384.77303944101 US Dollars
The average transaction cost was 280.7801770154955 US Dollars
The average depletion of capital for the transaction was 252.98246606986964 US Dollars per second
The average accumulation rate was 0.004509354725195056 Bitcoins per second
The standard deviation of accumulation rate was 0.013588551402010625 Bitcoins per second
Using Normal Approximation, the likelihood of accumulating 1 Bitcoin under 15 minutes is approximately 59.8738 %


In [23]:
BTC_USD_2021_sells

,Start Time,Start Nano,Arrival Price Millionths,End Time,End Nano,Accumulated Billionth,Notional Millionth,Transaction Cost Millionth,VWAP Millionths,Approximate Duration Time,Duration Nano,$\frac{Notional}{Time}$,$\frac{Accumulation}{Time}$
0,2021-04-10 16:28:52,515484000,59125070000,2021-04-10 16:45:53,173188000,1.004897e+09,5.939299e+10,2.969649e+08,59103585579,0 days 00:17:01,1020657704000,58.190899,0.000985
1,2021-04-10 16:45:55,582597000,59020970000,2021-04-10 17:05:14,434361000,1.061033e+09,6.269823e+10,3.134911e+08,59091676152,0 days 00:19:19,1158851764000,54.103753,0.000916
2,2021-04-10 17:05:15,495187000,59235090000,2021-04-10 17:19:00,275746000,9.259411e+08,5.490359e+10,2.745180e+08,59294907665,0 days 00:13:45,824780559000,66.567512,0.001123
3,2021-04-10 17:19:02,962543000,59273080000,2021-04-10 17:41:12,52492000,1.013241e+09,6.006940e+10,3.003470e+08,59284405397,0 days 00:22:10,1329089949000,45.195885,0.000762
4,2021-04-10 17:41:13,942379000,59234570000,2021-04-10 17:53:45,345825000,9.858224e+08,5.842456e+10,2.921228e+08,59264792327,0 days 00:12:32,751403446000,77.753917,0.001312
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2297,2021-04-26 17:35:04,40099000,53840720000,2021-04-26 18:01:08,815840000,9.987363e+08,5.354971e+10,2.677485e+08,53617464467,0 days 00:26:04,1564775741000,34.221971,0.000638
2298,2021-04-26 18:01:09,247929000,53530040000,2021-04-26 18:21:22,733919000,1.006636e+09,5.409822e+10,2.704911e+08,53741588140,0 days 00:20:13,1213485990000,44.580839,0.000830
2299,2021-04-26 18:21:26,198188000,53902960000,2021-04-26 18:45:11,723306000,9.965551e+08,5.372972e+10,2.686486e+08,53915449591,0 days 00:23:45,1425525118000,37.691176,0.000699
2300,2021-04-26 18:45:12,716958000,53900000000,2021-04-26 19:01:35,216487000,1.001876e+09,5.411851e+10,2.705926e+08,54017175323,0 days 00:16:23,982499529000,55.082480,0.001020


In [24]:
report_summary(BTC_USD_2021_sells, Order.Sell, 2021)

There are total of 2302 transactions in the year 2021 data where 1 Bitcoin was liquidated
The average duration time was 10.06131454360701 minutes
The average VWAP for the transaction was 56553.42043320765 US Dollars
The average transaction cost was 281.0282778970983 US Dollars
The average accumulation of capital for the transaction was 241.12108330467686 US Dollars per second
The average liquidation rate was 0.004341547618099131 Bitcoins per second
The standard deviation of liquidation rate was 0.021182895115809928 Bitcoins per second
Using Normal Approximation, the likelihood of liquidating 1 Bitcoin under 15 minutes is approximately 56.0605 %


<h3> 3-2. Year 2022 </h3>

In [25]:
# year 2022
BTC_USD_2022_buys = accumulation_analytics(BTC_USD_2022[Order.Buy.value - 1], one_bitcoin)
BTC_USD_2022_sells = accumulation_analytics(BTC_USD_2022[Order.Sell.value], one_bitcoin)

In [26]:
BTC_USD_2022_buys

,Start Time,Start Nano,Arrival Price Millionths,End Time,End Nano,Accumulated Billionth,Notional Millionth,Transaction Cost Millionth,VWAP Millionths,Approximate Duration Time,Duration Nano,$\frac{Notional}{Time}$,$\frac{Accumulation}{Time}$
0,2022-01-29 15:02:49,574850000,38391040000,2022-01-29 15:06:14,421260000,1.001364e+09,3.864137e+10,1.932069e+08,38588744458,0 days 00:03:25,204846410000,188.635835,0.004888
1,2022-01-29 15:06:14,885727000,38586660000,2022-01-29 15:16:27,240142000,1.010339e+09,3.898322e+10,1.949161e+08,38584300240,0 days 00:10:13,612354415000,63.661203,0.001650
2,2022-01-29 15:16:27,471207000,38506860000,2022-01-29 15:25:49,749557000,9.911882e+08,3.823859e+10,1.911929e+08,38578534579,0 days 00:09:22,562278350000,68.006510,0.001763
3,2022-01-29 15:25:51,346320000,38610940000,2022-01-29 15:40:53,719469000,9.960615e+08,3.839438e+10,1.919719e+08,38546193231,0 days 00:15:02,902373149000,42.548229,0.001104
4,2022-01-29 15:40:54,211425000,38460440000,2022-01-29 15:54:41,355155000,1.014694e+09,3.902484e+10,1.951242e+08,38459697612,0 days 00:13:47,827143730000,47.180238,0.001227
...,...,...,...,...,...,...,...,...,...,...,...,...,...
828,2022-02-04 18:36:24,495985000,41808960000,2022-02-04 18:42:46,134060000,9.911342e+08,4.143378e+10,2.071689e+08,41804410311,0 days 00:06:22,381638075000,108.568257,0.002597
829,2022-02-04 18:42:46,398025000,41861460000,2022-02-04 18:49:19,687273000,1.000655e+09,4.177980e+10,2.088990e+08,41752447135,0 days 00:06:33,393289248000,106.231745,0.002544
830,2022-02-04 18:49:19,960055000,41746660000,2022-02-04 18:57:10,640402000,9.996713e+08,4.157378e+10,2.078689e+08,41587450976,0 days 00:07:51,470680347000,88.326997,0.002124
831,2022-02-04 18:57:11,608690000,41553660000,2022-02-04 19:08:32,530073000,1.012165e+09,4.203851e+10,2.101925e+08,41533247225,0 days 00:11:21,680921383000,61.737685,0.001486


In [27]:
report_summary(BTC_USD_2022_buys, Order.Buy, 2022)

There are total of 833 transactions in the year 2022 data where 1 Bitcoin was accumulated
The average duration time was 10.674146916606643 minutes
The average VWAP for the transaction was 38164.794522929165 US Dollars
The average transaction cost was 190.75887853153023 US Dollars
The average depletion of capital for the transaction was 171.43638318598474 US Dollars per second
The average accumulation rate was 0.004452669968552648 Bitcoins per second
The standard deviation of accumulation rate was 0.014987281475721186 Bitcoins per second
Using Normal Approximation, the likelihood of accumulating 1 Bitcoin under 15 minutes is approximately 58.8217 %


In [28]:
BTC_USD_2022_sells

,Start Time,Start Nano,Arrival Price Millionths,End Time,End Nano,Accumulated Billionth,Notional Millionth,Transaction Cost Millionth,VWAP Millionths,Approximate Duration Time,Duration Nano,$\frac{Notional}{Time}$,$\frac{Accumulation}{Time}$
0,2022-01-29 15:02:50,23283000,38388540000,2022-01-29 15:14:51,140329000,1.000280e+09,3.858904e+10,1.929452e+08,38578239020,0 days 00:12:01,721117046000,53.512862,0.001387
1,2022-01-29 15:14:52,775836000,38582030000,2022-01-29 15:37:26,729526000,1.001022e+09,3.862117e+10,1.931058e+08,38581751631,0 days 00:22:34,1353953690000,28.524731,0.000739
2,2022-01-29 15:37:27,209172000,38449930000,2022-01-29 16:11:00,678797000,9.969107e+08,3.826433e+10,1.913216e+08,38382905212,0 days 00:33:33,2013469625000,19.004174,0.000495
3,2022-01-29 16:11:00,844807000,38217750000,2022-01-29 16:30:56,294873000,9.996020e+08,3.817602e+10,1.908801e+08,38191223102,0 days 00:19:56,1195450066000,31.934434,0.000836
4,2022-01-29 16:30:56,418648000,38093810000,2022-01-29 16:52:59,586721000,1.021489e+09,3.895059e+10,1.947530e+08,38131190664,0 days 00:22:03,1323168073000,29.437374,0.000772
...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,2022-02-04 18:49:58,850958000,41711000000,2022-02-04 18:52:18,613637000,9.793125e+08,4.077917e+10,2.038959e+08,41640615962,0 days 00:02:20,139762679000,291.774419,0.007007
596,2022-02-04 18:52:19,224701000,41581490000,2022-02-04 18:58:20,588169000,9.979174e+08,4.144640e+10,2.072320e+08,41532898226,0 days 00:06:01,361363468000,114.694495,0.002762
597,2022-02-04 18:58:20,767974000,41560430000,2022-02-04 19:02:31,559243000,9.659841e+08,4.011982e+10,2.005991e+08,41532590241,0 days 00:04:11,250791269000,159.972964,0.003852
598,2022-02-04 19:02:31,874681000,41514000000,2022-02-04 19:14:25,93772000,1.022984e+09,4.243142e+10,2.121571e+08,41478068179,0 days 00:11:54,713219091000,59.492824,0.001434


In [29]:
report_summary(BTC_USD_2022_sells, Order.Sell, 2022)

There are total of 600 transactions in the year 2022 data where 1 Bitcoin was liquidated
The average duration time was 14.815869459916666 minutes
The average VWAP for the transaction was 38238.969786935 US Dollars
The average transaction cost was 190.41803526407372 US Dollars
The average accumulation of capital for the transaction was 131.10868776570805 US Dollars per second
The average liquidation rate was 0.0034124438806649718 Bitcoins per second
The standard deviation of liquidation rate was 0.014735070174793268 Bitcoins per second
Using Normal Approximation, the likelihood of liquidating 1 Bitcoin under 15 minutes is approximately 56.2055 %


<h3> 3-3. Year 2023 </h3>

In [30]:
# year 2023
BTC_USD_2023_buys = accumulation_analytics(BTC_USD_2023[Order.Buy.value - 1], one_bitcoin)
BTC_USD_2023_sells = accumulation_analytics(BTC_USD_2023[Order.Sell.value], one_bitcoin)

In [31]:
BTC_USD_2023_buys

,Start Time,Start Nano,Arrival Price Millionths,End Time,End Nano,Accumulated Billionth,Notional Millionth,Transaction Cost Millionth,VWAP Millionths,Approximate Duration Time,Duration Nano,$\frac{Notional}{Time}$,$\frac{Accumulation}{Time}$
0,2023-01-23 18:47:48,676444000,22969900000,2023-01-23 18:54:34,953146000,1.022598e+09,2.348069e+10,1.174035e+08,22961794100,0 days 00:06:46,406276702000,57.794824,0.002517
1,2023-01-23 18:54:35,506670000,22965760000,2023-01-23 19:01:55,974552000,9.777328e+08,2.247199e+10,1.123599e+08,22983770961,0 days 00:07:20,440467882000,51.018444,0.002220
2,2023-01-23 19:01:56,168026000,23014310000,2023-01-23 19:08:28,649956000,1.169159e+09,2.690842e+10,1.345421e+08,23015193816,0 days 00:06:32,392481930000,68.559638,0.002979
3,2023-01-23 19:08:28,855920000,23020000000,2023-01-23 19:16:11,386215000,8.333199e+08,1.917026e+10,9.585131e+07,23004685185,0 days 00:07:43,462530295000,41.446500,0.001802
4,2023-01-23 19:16:12,223761000,22999840000,2023-01-23 19:25:51,628767000,1.003375e+09,2.306826e+10,1.153413e+08,22990666097,0 days 00:09:39,579405006000,39.813711,0.001732
...,...,...,...,...,...,...,...,...,...,...,...,...,...
282,2023-01-25 09:26:08,617279000,22495780000,2023-01-25 09:30:52,602429000,1.002703e+09,2.258872e+10,1.129436e+08,22527828004,0 days 00:04:44,283985150000,79.541906,0.003531
283,2023-01-25 09:30:52,726143000,22543310000,2023-01-25 09:34:41,966620000,1.000977e+09,2.258594e+10,1.129297e+08,22563887412,0 days 00:03:49,229240477000,98.525108,0.004366
284,2023-01-25 09:34:42,388759000,22547450000,2023-01-25 09:37:53,457134000,9.935272e+08,2.238985e+10,1.119493e+08,22535719182,0 days 00:03:11,191068375000,117.182397,0.005200
285,2023-01-25 09:37:54,87111000,22494070000,2023-01-25 09:41:53,319708000,1.003530e+09,2.258377e+10,1.129188e+08,22504328833,0 days 00:03:59,239232597000,94.400884,0.004195


In [32]:
report_summary(BTC_USD_2023_buys, Order.Buy, 2023)

There are total of 287 transactions in the year 2023 data where 1 Bitcoin was accumulated
The average duration time was 8.135263784552846 minutes
The average VWAP for the transaction was 22837.169702756095 US Dollars
The average transaction cost was 113.55669873796411 US Dollars
The average depletion of capital for the transaction was 96.75292864217708 US Dollars per second
The average accumulation rate was 0.004239074791192558 Bitcoins per second
The standard deviation of accumulation rate was 0.005208402480579953 Bitcoins per second
Using Normal Approximation, the likelihood of accumulating 1 Bitcoin under 15 minutes is approximately 72.5934 %


In [33]:
BTC_USD_2023_sells

,Start Time,Start Nano,Arrival Price Millionths,End Time,End Nano,Accumulated Billionth,Notional Millionth,Transaction Cost Millionth,VWAP Millionths,Approximate Duration Time,Duration Nano,$\frac{Notional}{Time}$,$\frac{Accumulation}{Time}$
0,2023-01-23 18:47:47,886114000,22969160000,2023-01-23 18:58:46,578432000,1.006825e+09,2.312179e+10,1.156089e+08,22965056486,0 days 00:10:59,658692318000,35.102564,0.001529
1,2023-01-23 18:58:49,744602000,22974230000,2023-01-23 19:08:46,716977000,9.938607e+08,2.286332e+10,1.143166e+08,23004554372,0 days 00:09:57,596972375000,38.298793,0.001665
2,2023-01-23 19:08:46,937856000,23020440000,2023-01-23 19:22:49,544896000,9.997368e+08,2.299001e+10,1.149500e+08,22996061094,0 days 00:14:03,842607040000,27.284378,0.001186
3,2023-01-23 19:22:52,745524000,22997820000,2023-01-23 19:37:17,896920000,1.007367e+09,2.313348e+10,1.156674e+08,22964302150,0 days 00:14:25,865151396000,26.739230,0.001164
4,2023-01-23 19:37:18,76147000,22976420000,2023-01-23 19:49:20,169860000,1.011425e+09,2.325001e+10,1.162500e+08,22987385888,0 days 00:12:02,722093713000,32.198050,0.001401
...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,2023-01-25 09:23:26,950269000,22486580000,2023-01-25 09:29:06,997550000,9.643343e+08,2.169945e+10,1.084972e+08,22501996698,0 days 00:05:40,340047281000,63.813029,0.002836
216,2023-01-25 09:29:11,147611000,22527560000,2023-01-25 09:33:50,495670000,1.009225e+09,2.276106e+10,1.138053e+08,22552997762,0 days 00:04:39,279348059000,81.479212,0.003613
217,2023-01-25 09:33:51,611808000,22567310000,2023-01-25 09:37:06,742254000,9.762767e+08,2.201547e+10,1.100774e+08,22550445881,0 days 00:03:15,195130446000,112.824397,0.005003
218,2023-01-25 09:37:06,932345000,22512230000,2023-01-25 09:41:44,992813000,9.590422e+08,2.157875e+10,1.078937e+08,22500313271,0 days 00:04:38,278060468000,77.604521,0.003449


In [34]:
report_summary(BTC_USD_2023_sells, Order.Sell, 2023)

There are total of 220 transactions in the year 2023 data where 1 Bitcoin was liquidated
The average duration time was 10.615273405151516 minutes
The average VWAP for the transaction was 22822.278964013636 US Dollars
The average transaction cost was 114.25935418048329 US Dollars
The average accumulation of capital for the transaction was 104.01968790776594 US Dollars per second
The average liquidation rate was 0.004560536891532884 Bitcoins per second
The standard deviation of liquidation rate was 0.01813937669805956 Bitcoins per second
Using Normal Approximation, the likelihood of liquidating 1 Bitcoin under 15 minutes is approximately 57.5409 %
